In [2]:
# 第一次运行需要加代理下载huggingface权重
from PIL import Image
from surya.ocr import run_ocr
from surya.model.detection.segformer import load_model as load_det_model
from surya.model.detection.segformer import load_processor as load_det_processor
from surya.model.recognition.model import load_model as load_rec_model
from surya.model.recognition.processor import load_processor as load_rec_processor
from surya.model.ordering.processor import load_processor as load_ordering_processor
from surya.model.ordering.model import load_model as load_ordering_model
from surya.settings import settings

# det_processor, det_model = load_det_processor(checkpoint=settings.DETECTOR_MATH_MODEL_CHECKPOINT), load_det_model(checkpoint=settings.DETECTOR_MATH_MODEL_CHECKPOINT)
det_processor, det_model = load_det_processor(), load_det_model()
rec_model, rec_processor = load_rec_model(), load_rec_processor()
layout_model = load_det_model(checkpoint=settings.LAYOUT_MODEL_CHECKPOINT)
layout_processor = load_det_processor(checkpoint=settings.LAYOUT_MODEL_CHECKPOINT)
order_model = load_ordering_model()
order_processor = load_ordering_processor()

/hf_shared/hfai_envs/wangjiawei/wjw_surya_0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/hf_shared/hfai_envs/wangjiawei/wjw_surya_0/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/hf_shared/hfai_envs/wangjiawei/wjw_surya_0/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded detection model vikp/surya_det2 on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec on device cuda with dtype torch.float16
Loaded detection model vikp/surya_layout2 on device cuda with dtype torch.float16
Loaded reading order model vikp/surya_order on device cuda with dtype torch.float16


{0: 'Blank',
 1: 'Caption',
 2: 'Footnote',
 3: 'Formula',
 4: 'List-item',
 5: 'Page-footer',
 6: 'Page-header',
 7: 'Picture',
 8: 'Section-header',
 9: 'Table',
 10: 'Text',
 11: 'Title'}

In [25]:
# IMAGE_PATH = "/weka-jd/prod/deepseek/permanent/wangjiawei/workspace/surya/test_data/04194-00/26194194_0026.input.png"
IMAGE_PATH = "/weka-jd/prod/deepseek/permanent/wangjiawei/workspace/surya/test_data/10015308/14_1.png"
image = Image.open(IMAGE_PATH)
langs = ["en", "zh"] # Replace with your languages

In [26]:
from surya.ocr import run_ocr
from surya.detection import batch_text_detection
from surya.layout import batch_layout_detection
from surya.ordering import batch_ordering
import copy

ocr_predictions = run_ocr([image], [langs], det_model, det_processor, rec_model, rec_processor)
text_line_predictions = batch_text_detection([image], det_model, det_processor)
layout_predictions = batch_layout_detection([image], layout_model, layout_processor, copy.deepcopy(text_line_predictions))

# bboxes = [line_bbox.bbox for i, line_bbox in enumerate(ocr_predictions[0].text_lines)]
bboxes = [layout_box.bbox for i, layout_box in enumerate(layout_predictions[0].bboxes)]
order_predictions = batch_ordering([image], [bboxes], order_model, order_processor)

Finding reading order: 100%|██████████| 1/1 [00:00<00:00,  3.74it/s]


In [27]:
# transform the predictions of ocr, layout, order into the format of the output
"""
{
  "text": "<deepseek_image>\nEilish was finding it hard to breathe and closed her eyes.\nAnya had returned to her place in the circle next to her sister. \"Are you all right?\"\nEilish shook her head no.\nAnya grabbed her hand and squeezed. \"Try to hold on.\"\nFina began her call to the spirits. She rang the bell three times and stood regal before the small fire, her breasts bare, her thick dark hair falling to her shoulders. She raised both hands to the heavens as four warlocks in their hooded robes broke from the circle to approach the frightened stag. They had drugged the deer so as not to have him panic, but he struggled nonetheless as the warlocks deftly cut his throat, and the blood of the sacrificed animal ran in a pool beneath Fina's feet.\nEilish began to hyperventilate and fought the pull into the circle’s center.\nFina's voice rose strong, \"On this night, all roads cross, all gates open. All nights, all dreams, all worlds have come together at the death of the old year. On this night of ancients, the gates to the other world are opened wide, and the veil grows thin. We seek guidance from those who walked before us.\"\nThe men had severed the antlers from the now-dead buck and were raising it to place on Fina's head when Eilish felt herself dropping into an unconscious state, and the spirit of Rhiannon entered her. The winds picked up as she broke from the circle, holding her own hands skyward, her head back as she shouted. \"I invoke you, Hekate. You of roads and crossways, Goddess of the Heavens, the Underworld, and the sea. You who among the tombs, dancing with the dead souls. Respond to the daughter of Rhiannon.\"\nFina turned and looked on in horror. What was she doing? This wasn't part of the ritual! She looked at Warrick, who shook his head, sensing a power he didn't wish to confront in Eilish, at least not in front of the coven. The men carrying the antlers stopped in their tracks as Eilish approached, and Fina backed away. The men looked at each other in confusion but proceeded to secure the antlers to Eilish's head. The coven was taken aback by this change in the ritual and momentarily stopped their drumming as they watched in awe as the antlers were placed on Eilish, and her snow-white hair slowly turned red as the blood of the buck ran through her hair and down her face, dripping upon her breast.\n<deepseek_image>\nBoth Ian and Carter had been quiet, but their eyes were glued to the drama playing out before them. Carter seemed mesmerized by what was happening, his eyes never leaving Anya.\nSuddenly Ian stood up in the tree, and Carter looked at him. \"What’s wrong? This is some freak show, to begin with, but when you get rattled, I get anxious.\" Ian felt his heart hammering in his chest when Eilish entered a semi-conscious state and felt the entity of the being that entered her. His beast roared inside, but he knew he could not step in. \"This isn’t right; something is happening to Eilish.\"\nCarter swung his head back to the ritual in progress and saw the antlers on her head, blood dripping. \"Fuck me, bro, this is like something satanic. I don’t like this. I don’t like Anya being down there in the middle of this.\"\nIan whispered. \"I feel the same, Carter, but here’s the deal. It’s not satanic; this ritual is about honoring the harvest and bounty of a good year. It’s as old as time. Your child will be a part of this. And that woman who was leading is your soon-to-be Mother-in-law. Anya will teach you. But I think Eilish just changed up the rules of this game and things are about to ramp up.\"\nCarter's head was spinning. He hadn't thought about the ramifications of having a child with a witch. Of course, the baby would be taught these rituals and have their power. He was beginning to understand why the coven wouldn’t receive him with open arms as he listened to Ian's explanation.\n",
  "images": [
    "input.png",
    "image_0000.png"
  ],
  "similarities": [
    1.0,
    1.0
  ],
  "type": "interleave",
  "image_tag": "<deepseek_image>",
  "template": "DOUBLE-DOUBLE"
}
"""


from collections import defaultdict
import os

# IMAGE_PATH = "/weka-jd/prod/deepseek/permanent/wangjiawei/workspace/surya/test_data/04194-00/26194194_0026.input.png"
# IMAGE_PATH = "/weka-jd/prod/deepseek/permanent/wangjiawei/workspace/surya/test_data/10015308/14_1.png"
image = Image.open(IMAGE_PATH)
langs = ["en", "zh"] # Replace with your languages

save_path = "/weka-jd/prod/deepseek/permanent/wangjiawei/workspace/surya/test_data/test"
image_name = IMAGE_PATH.split("/")[-1]

image.save(os.path.join(save_path, image_name))

output = {
    "text": "<deepseek_image>\n",
    "images": ['input.png'],
    "similarities": [1.0],
    "type": "", # Document
    "image_tag": "<deepseek_image>",
    "template": "" # I don't know what the key of "template" does mean
}

# Firstly, map the ocr id to the layout id
layout_id_to_ocr_id = defaultdict(list)
ocr_id_to_layout_id = {}
for i, ocr_line in enumerate(ocr_predictions[0].text_lines):
    for j, layout_line in enumerate(layout_predictions[0].bboxes):
        if ocr_line.intersection_pct(layout_line) > 0.9:
            ocr_id_to_layout_id[i] = j
            layout_id_to_ocr_id[j].append(i)
            break

order = [b.position for b in order_predictions[0].bboxes]
# Then, sort the layout id by the order
layout_ids = [order.index(i) for i in range(len(order))]

# Finally, generate the output
additional_image_id = 0
for layout_id in layout_ids:
    ocr_ids = layout_id_to_ocr_id[layout_id]
    layout_pred = layout_predictions[0].bboxes[layout_id]
    label = layout_pred.label
    # layout_model.config.id2label
    """
    {0: 'Blank',
    1: 'Caption',
    2: 'Footnote',
    3: 'Formula',
    4: 'List-item',
    5: 'Page-footer',
    6: 'Page-header',
    7: 'Picture',
    8: 'Section-header',
    9: 'Table',
    10: 'Text',
    11: 'Title'}
    """
    if label not in ['Picture', 'Figure', 'Table']:
        for ocr_id in ocr_ids:
            # Two options here: append "\n" at the end of each text-line or append "\n" at the end of each paragraph (may be not precious)
            # output["text"] += ocr_predictions[0].text_lines[ocr_id].text + "\n" 
            output["text"] += ocr_predictions[0].text_lines[ocr_id].text
        output["text"] += "\n"
    elif label in ['Picture', 'Figure']:
        # need to crop the image and save it as a new image

        # crop the image
        image = Image.open(IMAGE_PATH)
        cropped_image = image.crop(layout_pred.bbox)
        cropped_image.save(os.path.join(save_path,f"{image_name.split('.')[0]}.image_{additional_image_id:04d}.png"))

        output["text"] += "<deepseek_image>\n"
        output["images"].append(f"image_{additional_image_id:04d}.png")
        output["similarities"].append(1.0)

        additional_image_id += 1

        for ocr_id in ocr_ids:
            output["text"] += ocr_predictions[0].text_lines[ocr_id].text + "\n"
    elif label == "Table":
        # Take table in document as an image unless we have a tool to convert the table into a structured data
        # need to crop the image and save it as a new image
        # crop the image
        image = Image.open(IMAGE_PATH)
        cropped_image = image.crop(layout_pred.bbox)
        cropped_image.save(os.path.join(save_path,f"{image_name.split('.')[0]}.image_{additional_image_id:04d}.png"))

        output["text"] += "<deepseek_image>\n"
        output["images"].append(f"image_{additional_image_id:04d}.png")
        output["similarities"].append(1.0)

        additional_image_id += 1

        for ocr_id in ocr_ids:
            output["text"] += ocr_predictions[0].text_lines[ocr_id].text + "\n"
    else:
        raise ValueError(f"Unknown label {label}")
    
# save the output into a json file
# there are many chinese word
# so we need to save the file with utf-8
import json
with open(os.path.join(save_path,f"{image_name.split('.')[0]}.text.json"), "w", encoding="utf-8") as f:
    json.dump(output, f, indent=4)

In [28]:
from PIL import Image, ImageDraw
# Visualize the Layout Results in Original Image (need to handcraft)

# create an image copy
image_c = image.copy()
# 创建一个 ImageDraw 对象
draw = ImageDraw.Draw(image_c)
# draw the bounding boxes
for i, layout_bbox in enumerate(layout_predictions[0].bboxes):
    bbox = layout_bbox.bbox
    label = layout_bbox.label
    draw.rectangle(bbox, outline="red", width=3)
    draw.text((bbox[0], bbox[1]), label, fill="red")

# Visualize the OCR Results (text_line) in Original Image (need to handcraft)

for i, line_bbox in enumerate(ocr_predictions[0].text_lines):
    bbox = line_bbox.bbox
    text = line_bbox.text
    draw.rectangle(bbox, outline="green", width=3)

# Visualize the Reading Order Results in Original Image (need to handcraft)
# sort the bboxes based on the order
order_boxes = order_predictions[0].bboxes
order_boxes = sorted(order_boxes, key=lambda x: x.position, reverse=False)
for i, order_bbox in enumerate(order_boxes):
    bbox = order_bbox.bbox
    draw.rectangle(bbox, outline="blue", width=3)
    # draw arrow from the center of previous bbox to that of current bbox
    if i > 0:
        pre_center = ((order_boxes[i-1].bbox[0] + order_boxes[i-1].bbox[2])/2, (order_boxes[i-1].bbox[1] + order_boxes[i-1].bbox[3])/2)
        cur_center = ((order_boxes[i].bbox[0] + order_boxes[i].bbox[2])/2, (order_boxes[i].bbox[1] + order_boxes[i].bbox[3])/2)
        draw.line([pre_center, cur_center], fill="blue", width=3)
    

image_c.save(os.path.join(save_path,f'{IMAGE_PATH.split("/")[-1].split(".")[0]}_layout.png'))

In [59]:
ocr_predictions[0].text_lines[0].bbox

[98.0, 307.0, 1848.0, 339.0]

In [80]:
order_boxes = order_predictions[0].bboxes
order_boxes = sorted(order_boxes, key=lambda x: x.position, reverse=False)

In [46]:
ocr_predictions[0].text_lines[1]

TextLine(polygon=[[32.0, 83.0], [244.0, 83.0], [244.0, 100.0], [32.0, 100.0]], confidence=0.99072265625, text='that’s settled then! Let’s dig in."', bbox=[32.0, 83.0, 244.0, 100.0])